In [1]:
import os
import time
import random

import numpy as np
import pandas as pd

from tqdm import trange, tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score

np.random.seed(666)

In [2]:
def prettify_duration(duration_seconds):
    s = duration_seconds % 60
    duration_in_minutes = (duration_seconds - s) // 60
    m = duration_in_minutes % 60
    h = (duration_in_minutes - m) // 60

    return f"{h}h {m}m {s}s"

In [3]:
X_train_df = pd.read_table(os.path.join(os.getcwd(), "data", "x_train.txt"), header=None, sep=" ")
y_train_df = pd.read_table(os.path.join(os.getcwd(), "data", "y_train.txt"), header=None)
X_test_df = pd.read_table(os.path.join(os.getcwd(), "data", "x_test.txt"), header=None, sep=" ")

In [4]:
X_train = X_train_df.values
X_train_scaled = StandardScaler().fit_transform(X_train)
y_train = y_train_df.values.flatten()
X_test = X_test_df.values

In [5]:
def calculate_reward(model, X, y):
    probabilities_of_label_1 = model.predict_proba(X)[:, 1]
    s = np.argsort(probabilities_of_label_1)
                  
    income = (np.round(probabilities_of_label_1[s[-1000:]]) == y[s[-1000:]]).sum() * 10
    cost = model.n_features_in_ * 200

    return income - cost

In [6]:
class Lasso:
    def __init__(self, X, y):
        self.stdsc = StandardScaler().fit(X)
        
        self.X = self.stdsc.transform(X)
        self.y = y
        
        self.best_C = None
        self.best_features = None

        return

    def fit(self):
        best_reward = float("-inf")
        selected_features = np.arange(self.X.shape[1])
        
        for C in tqdm(np.linspace(1, 0.001, 1000)):
            model = LogisticRegression(penalty="l1", solver="liblinear", C=C).fit(self.X[:, selected_features], self.y)

            reward = calculate_reward(model, self.X[:, selected_features], self.y)
        
            selected_features = np.argwhere(model.coef_.flatten() != 0).flatten()
            
            if best_reward < reward:
                best_reward = reward
                
                self.best_C = C
                self.best_features = selected_features

        return self

    def get_best_model(self):
        return LogisticRegression(penalty="l1", solver="liblinear", C=self.best_C).fit(self.X[:, self.best_features], self.y)

    def predict(self, X):
        X = self.stdsc.transform(X)
        model = self.get_best_model()

        probabilities_of_label_1 = model.predict_proba(X[:, self.best_features])[:, 1]
        s = np.argsort(probabilities_of_label_1)

        return s[-1000:]        

In [7]:
lasso = Lasso(X_train, y_train).fit()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [05:19<00:00,  3.13it/s]


In [8]:
lasso.best_features

array([0, 1])

In [9]:
pd.DataFrame(lasso.best_features).to_csv("solutions/lasso_features_selected.csv")

In [10]:
lasso_best_model = lasso.get_best_model()

In [11]:
calculate_reward(lasso_best_model, X_train_scaled[:, lasso.best_features], y_train)

np.int64(6640)

In [12]:
accuracy_score(y_train, lasso_best_model.predict(X_train_scaled[:, lasso.best_features]))

0.6456

In [13]:
precision_score(y_train, lasso_best_model.predict(X_train_scaled[:, lasso.best_features]))

0.651194231635872

In [14]:
recall_score(y_train, lasso_best_model.predict(X_train_scaled[:, lasso.best_features]))

0.5914858780188293

In [15]:
lasso_X_test_pred = lasso.predict(X_test)
lasso_X_test_pred[:10]

array([1755, 1651,  978, 4963, 4305, 3505, 3537, 4248, 4744, 4255])

In [16]:
pd.DataFrame(lasso_X_test_pred).to_csv("solutions/lasso_observations_predicted.csv")

In [17]:
class GA:
    def __init__(self, X, y):
        self.stdsc = StandardScaler().fit(X)
        
        self.X = self.stdsc.transform(X)
        self.y = y
        
        self.target_indexes = np.argwhere(self.y == 1)
        
        return

    def generate_population(self, population_size):
        return [np.random.choice(self.X.shape[1], np.random.randint(1, self.X.shape[1] + 1), False) for _ in range(population_size)]

    def evaluate_features(self, features):
        model = LogisticRegression(penalty=None).fit(self.X[:, features], self.y)

        reward = calculate_reward(model, self.X[:, features], self.y)
        
        return -reward

    def get_best_model(self, features):
        return LogisticRegression(penalty=None).fit(self.X[:, features], self.y)

    def cross_over(self, parent_a, parent_b):
        all_indexes = np.unique(np.concatenate((parent_a,parent_b)))
        
        return (
            np.random.choice(all_indexes, np.random.randint(1, len(all_indexes) + 1), False),
            np.random.choice(all_indexes, np.random.randint(1, len(all_indexes) + 1), False)
        )

    def mutate(self, representative):
        to_remove = np.random.choice(self.X.shape[1], np.random.randint(1, 5 + 1), False)
        to_add = np.random.choice(self.X.shape[1], np.random.randint(1, 5 + 1), False)
        
        return np.unique(np.union1d(np.setdiff1d(representative, to_remove), to_add))

    def run(self, population_size=100, number_of_generations=100, cross_over_prob=0.2, mutation_prob=0.2):
        population = self.generate_population(population_size)
    
        for generation in tqdm(range(number_of_generations)):
            ## cross-over
            # calculate values of function to optimise
            proximities = np.array([self.evaluate_features(features) for features in population])
            # and transform them into probabilities
            probabilities = self.proximities2probabilities(proximities)
            children = []
            # try to perform cross-over population_size times
            for _ in range(population_size):
                # if cross-over chance is successful
                if random.random() < cross_over_prob:
                    # select two parents for cross-over
                    parent_indexes = np.random.choice(len(population), 2, False, probabilities)
                    # retrieve parents
                    parent_a, parent_b = population[parent_indexes[0]], population[parent_indexes[1]]
                    # perform cross-over
                    child_a, child_b = self.cross_over(parent_a, parent_b)
                    children.append(child_a)
                    children.append(child_b)
            
            population.extend(children)
            
            ## mutation
            # calculate values of function to optimise
            proximities = np.array([self.evaluate_features(features) for features in population])
            # and transform them into probabilities
            probabilities = self.proximities2probabilities(proximities)
            mutated = []
            # try to perform mutation population_size times
            for _ in range(population_size):
                # if mutation chance is successful
                if random.random() < mutation_prob:
                    # select one representative to be mutated
                    index = np.random.choice(len(population), 1, False, probabilities)
                    # retrieve representative
                    representative = population[index[0]]
                    # perform mutation
                    mutant = self.mutate(representative)
                    mutated.append(mutant)
                    
            population.extend(mutated)
    
            ## selection
            # calculate values of function to optimise
            proximities = np.array([self.evaluate_features(features) for features in population])
            # and transform them into probabilities
            probabilities = self.proximities2probabilities(proximities)
            # evaluate, how much is 10% of population_size
            top_10_best_len = int(0.1 * population_size)
            # get indexes of models from population sorted by values of optimised function
            s = np.argsort(proximities)
            # top 10% of population_size models are advancing to new generation by default
            new_population = [population[idx] for idx in s[:top_10_best_len]]
            # rest indexes select randomly from current population
            pr = probabilities[s[top_10_best_len:]]
            rest_indexes = np.random.choice(s[top_10_best_len:], population_size - top_10_best_len, False, pr / pr.sum())
            # fill new_population up to population_size elements
            new_population.extend([population[idx] for idx in rest_indexes])
            # replace current population with new one
            population = new_population
    
        return population

    def predict(self, X, features):
        X = self.stdsc.transform(X)
        model = self.get_best_model(features)
        
        probabilities_of_label_1 = model.predict_proba(X[:, features])[:, 1]
        s = np.argsort(probabilities_of_label_1)
        
        return s[-1000:]

    @staticmethod
    def proximities2probabilities(proximities):
        proximities = np.clip(proximities, -50, 50)
        mods = np.exp(-proximities)
    
        return mods / mods.sum()

In [18]:
ga = GA(X_train, y_train)
population = ga.run()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:29<00:00,  2.09s/it]


In [19]:
population[0]

array([8], dtype=int32)

In [20]:
pd.DataFrame(population[0]).to_csv("solutions/ga_features_selected.csv")

In [21]:
ga_best_model = ga.get_best_model(population[0])

In [22]:
calculate_reward(ga_best_model, X_train_scaled[:, population[0]], y_train)

np.int64(7210)

In [23]:
accuracy_score(y_train, ga_best_model.predict(X_train_scaled[:, population[0]]))

0.653

In [24]:
precision_score(y_train, ga_best_model.predict(X_train_scaled[:, population[0]]))

0.6614963503649635

In [25]:
recall_score(y_train, ga_best_model.predict(X_train_scaled[:, population[0]]))

0.5935325419566108

In [26]:
ga_X_test_pred = ga.predict(X_test, population[0])
ga_X_test_pred[:10]

array([4230, 3567, 3351,  200, 1229, 1200,  977, 4243, 3317, 2406])

In [27]:
pd.DataFrame(ga_X_test_pred).to_csv("solutions/ga_observations_predicted.csv")